In [1]:
import os
import glob
from itertools import product

import numpy as np
import pandas as pd

import cv2
from skimage.metrics import structural_similarity as ssim

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def create_dirs(run):    
    output_dir= os.getcwd() + f"\\spatial-attention-complete_ssim\\run-{run}"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    return output_dir

In [3]:
def struct_sim(run, chunk_list):
    """
    Read images according to event file
    """
    event_input = pd.read_csv(os.getcwd() + "\\reference_spatial-attention" + f"\\complete_ses-movie_task-movie_run-{run}_events.tsv", sep="\t", index_col=False)

    ssim_list, chunk_max = [], chunk_list[int(run)]
    
    for onset_a, onset_b in product(event_input["onset"], repeat=2):
        input_img_path_a = glob.glob(os.getcwd() + "\qc_frames" + f"\*run-{run}" + f"\*run-{run}*onset-{int(onset_a)}.0.jpg")
        input_img_path_b = glob.glob(os.getcwd() + "\qc_frames" + f"\*run-{run}" + f"\*run-{run}*onset-{int(onset_b)}.0.jpg")

        gray1 = cv2.cvtColor(cv2.imread(input_img_path_a[0]), cv2.COLOR_BGR2GRAY)
        gray2 = cv2.cvtColor(cv2.imread(input_img_path_b[0]), cv2.COLOR_BGR2GRAY)  
        ssim_list.append(ssim(gray1, gray2))
    return np.reshape(ssim_list, (chunk_max, chunk_max))

In [4]:
def plot_heatmap(run, labels, df, output_heat_maps):
    # masked_tria = np.triu(df)
    hm_fig_dissim = sns.heatmap(df, xticklabels=labels, yticklabels=labels, 
                         cmap="RdBu_r") # , mask=masked_tria
    hm_fig_dissim.set_title(f"run-{run}", fontweight="bold")
    plt.savefig(output_heat_maps + f"\\ssim_run-{run}.png", bbox_inches="tight", pad_inches=0)
    plt.close()
    return None

In [5]:
chunk_list = [0, 253, 252, 242, 275, 249, 243, 306, 178]

In [6]:
for run in range(1, 9):
    print(f"starting run-{run}")
    output_dir = create_dirs(run)
    struct_sim_df = struct_sim(run, chunk_list)
    plot_heatmap(run, list(range(chunk_list[int(run)])), struct_sim_df, output_dir)
    pd.DataFrame(struct_sim_df).to_csv(output_dir + f"\\ssim_run-{run}.tsv", header=False, index=False)    

starting run-6
starting run-7
starting run-8
